In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

cuda_available = torch.cuda.is_available()
print("CUDA available:", cuda_available)


CUDA available: True


In [2]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, activation):
        super(MLP, self).__init__()
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.activation = activation

        self.fc_layers = nn.ModuleList()
        prev_size = input_size
        for hidden_size in hidden_sizes:
            self.fc_layers.append(nn.Linear(prev_size, hidden_size))
            prev_size = hidden_size

        self.output_layer = nn.Linear(prev_size, output_size)

    def forward(self, x):
        x = x.view(-1, self.input_size)
        for fc_layer in self.fc_layers:
            x = self.activation(fc_layer(x))
        x = self.output_layer(x)
        return x

In [1]:
from sklearn.datasets import fetch_california_housing
import pandas as pd

california = fetch_california_housing()
X = california.data
y = california.target

# Create a dataframe with the input and output arrays
df = pd.DataFrame(X, columns=california.feature_names)
df['Target'] = y
# df = df.drop(columns=df.columns[-2])

# Save the dataframe to a CSV file
df.to_csv('data.csv', index=False)

# Print the shape of the input and output arrays
print("Input shape:", X.shape)
print("Output shape:", y.shape)
# print(y[0])

Input shape: (20640, 8)
Output shape: (20640,)


/tmp/ipykernel_5281/91945156.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [7]:
from sklearn.model_selection import train_test_split
import numpy as np
import time

if torch.cuda.is_available():
    device = torch.device('cuda')
    print('Using GPU')
else:
    device = torch.device('cpu')
    print('Using CPU')

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the data into PyTorch tensors and move them to the GPU
X_train = torch.FloatTensor(X_train).to(device)
X_test = torch.FloatTensor(X_test).to(device)
y_train = torch.FloatTensor(y_train).to(device)
y_test = torch.FloatTensor(y_test).to(device)

# Define the model architecture and move it to the GPU
input_size = X_train.shape[1]
hidden_sizes = [16,4,16]
try:
    output_size = y_train.shape[1]
except IndexError:
    output_size = 1
model = MLP(input_size, hidden_sizes, output_size, F.relu).to(device)

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define the loss function
criterion = nn.MSELoss()

# Number of epochs
n_epochs = 100

# Placeholder for losses
train_losses = np.zeros(n_epochs)
test_losses = np.zeros(n_epochs)

# Get the current time before the for loop
start_time = time.time()

for it in range(n_epochs):
    # zero the parameter gradients
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward and optimize
    loss.backward()
    optimizer.step()

    # Save losses
    train_losses[it] = loss.item()

    # Test loss
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, y_test)
    test_losses[it] = test_loss.item()

    if (it + 1) % 5 == 0:
        print(f'Epoch {it+1}/{n_epochs}, Train Loss: {loss.item():.4f}, Test Loss: {test_loss.item():.4f}')

# Get the current time after the for loop
end_time = time.time()

# Calculate the time difference
time_difference = end_time - start_time

# Print the time difference
print("Time difference:", time_difference)


Using GPU
Epoch 5/100, Train Loss: 2.8231, Test Loss: 2.4201
Epoch 10/100, Train Loss: 2.2540, Test Loss: 2.1145
Epoch 15/100, Train Loss: 1.7189, Test Loss: 1.6877
Epoch 20/100, Train Loss: 1.4624, Test Loss: 1.4059
Epoch 25/100, Train Loss: 1.4224, Test Loss: 1.4274
Epoch 30/100, Train Loss: 1.4341, Test Loss: 1.4234
Epoch 35/100, Train Loss: 1.4356, Test Loss: 1.4047
Epoch 40/100, Train Loss: 1.4175, Test Loss: 1.3750
Epoch 45/100, Train Loss: 1.3984, Test Loss: 1.3680
Epoch 50/100, Train Loss: 1.3919, Test Loss: 1.3686
Epoch 55/100, Train Loss: 1.3889, Test Loss: 1.3583
Epoch 60/100, Train Loss: 1.3810, Test Loss: 1.3519
Epoch 65/100, Train Loss: 1.3754, Test Loss: 1.3491
Epoch 70/100, Train Loss: 1.3716, Test Loss: 1.3429
Epoch 75/100, Train Loss: 1.3670, Test Loss: 1.3393
Epoch 80/100, Train Loss: 1.3638, Test Loss: 1.3368
Epoch 85/100, Train Loss: 1.3607, Test Loss: 1.3338
Epoch 90/100, Train Loss: 1.3584, Test Loss: 1.3312
Epoch 95/100, Train Loss: 1.3563, Test Loss: 1.3295
Epo